In [21]:
from itertools import cycle

import black
import jupyter_black


jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [24]:
# Day 1: Chronal Calibration
def first_repeat(frequencies):
    freqs = cycle(iter(frequencies))
    found = set()
    current = 0
    while current not in found:
        found.add(current)
        current += next(freqs)
    return current


lines = open("2018/1.txt").read().splitlines()
frequencies = [int(line) for line in lines]
print(f"Part 1: {sum(frequencies)}") # 411
print(f"Part 2: {first_repeat(frequencies)}") # 56360

Part 1: 411
Part 2: 56360
